In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

2024-02-03 10:15:59.778737: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Extrait les dernières données et écrit le dataset

In [2]:
from lib.dataeng import Etl

In [3]:
etl = Etl()

df_weight_raw, df_food_raw, df_exos_raw, df_all = etl.extract_store_all()


Le fichier poids le plus récent est : data_BEN_1706944407.zip, parmi :
data_BEN_1706593344.zip
data_BEN_1706676305.zip
data_BEN_1706944407.zip
data_BEN_1706861557.zip
data_BEN_1706761334.zip

Le fichier food le plus récent est : File-Export-2020-08-03-to-2024-02-03.zip, parmi
File-Export-2020-08-03-to-2024-02-01.zip
File-Export-2020-08-03-to-2024-02-03.zip
File-Export-2020-08-03-to-2024-01-31.zip
File-Export-2020-08-03-to-2024-02-02.zip
File-Export-2020-08-03-to-2024-01-30.zip

Le fichier exercices le plus récent est : polar-user-data-export_703c11e2-d0c0-42d0-84a6-665ce3a9f847.zip, parmi :
polar-user-data-export_12baa4ce-79a5-40f8-ad40-74308f7c2c26.zip
polar-user-data-export_703c11e2-d0c0-42d0-84a6-665ce3a9f847.zip


100%|██████████| 1868/1868 [01:33<00:00, 19.96it/s]


In [4]:
fullds_path = '/home/benjamin/Folders_Python/Weight/data/full_dataset.csv'

with open(fullds_path, 'r') as f:
    df_full = pd.read_csv(f, index_col=0)
    
cols_to_drop = ['Masse_Osseuse', 'Masse_Musculaire', 'Masse_Hydrique']
df_full.drop(columns=cols_to_drop, inplace=True)
df_full.dropna(inplace=True)

In [5]:
df_full.describe(include='all').transpose()

,count,mean,std,min,25%,50%,75%,max
Masse_Totale,1256.0,73.737698,7.768499,66.000000,69.265625,71.352500,73.307604,107.250000
Masse_Grasse,1256.0,12.189656,5.156837,7.541667,8.939154,10.840875,12.167619,37.190000
MG%,1256.0,0.160802,0.043915,0.112927,0.128909,0.151548,0.166688,0.346760
BMR,1256.0,1699.437706,59.228839,1626.348571,1664.184250,1679.345200,1705.772800,1933.516000
Calories,1256.0,1967.675557,458.060366,0.300000,1780.300000,1850.350000,2038.900000,5762.000000
Lipides,1256.0,52.834793,22.363461,0.000000,38.600000,49.300000,64.000000,235.900000
Glucides,1256.0,213.088854,75.604967,0.100000,168.675000,199.100000,240.925000,875.400000
Proteines,1256.0,157.093392,39.920517,0.000000,133.900000,162.050000,184.500000,267.100000
exo_duree,1256.0,2695.534215,2798.140951,0.000000,0.000000,2326.293000,4744.896000,28800.000000
exo_cals_bruts,1256.0,295.397293,295.526617,0.000000,0.000000,275.500000,497.250000,2290.000000


In [19]:
ds = df_full[['Calories', 'exo_cals_nets', 'Masse_Totale', 'Masse_Grasse']]

In [20]:
ds

,Calories,exo_cals_nets,Masse_Totale,Masse_Grasse
2020-08-09,1861.3,0.000000,107.250000,37.190000
2020-08-11,1772.4,0.000000,104.650000,33.150000
2020-08-12,1894.3,0.000000,104.350000,33.800000
2020-08-13,1822.6,0.000000,104.400000,34.340000
2020-08-14,1891.5,0.000000,104.500000,33.850000
...,...,...,...,...
2024-01-29,1795.7,181.483944,73.896667,12.780000
2024-01-30,1680.9,0.000000,73.340000,12.521538
2024-01-31,1682.0,332.372089,73.571538,12.973077
2024-02-01,1664.6,160.687639,73.963333,12.926667


In [21]:
# from lib.librnn import CreateDataset  # utilisation de tf.dataset : compliqué
# essai = CreateDataset(ds)
# essai.dataframe.head(20)

# dsf = essai.create_dataset()

### création dataset

On créé un dataset pour des modèles de sequence-to-sequence.

Hyperparamètres :
- PAST = longueur (en jours) de l'historique que l'on va prendre en entrée. Détermine la longueur de la séquence en entrée.
- FUTURE = longueur (en jours) de la fenêtre sur laquelle on veut prédire. Longueur de la séquence en sortie

Description :
- INPUTS : au timestep d'indice i, c'est une séquence (vecteur) concaténation de:
  - la séquence de longueur PAST, indices de i-PAST+1 à i (inclus) des features exo_cals_nets, Calories, Masse_Totale, Masse_Grasse
  - la séquence de longueur FUTURE, indices de i+1 à i+FUTURE (inclus) des features exo_cals_nets, Calories
- OUTPUTS : les valeurs cibles au timestep d'indce sont une séquence (vecteur):
  - séquence de longueur FUTURE, formée des indices i+1 à i+FUTURE des valeurs Masse_Totale, Masse_Grasse

Le modèle vont apprendre sur des séquences formées de tous les inputs passés (calories exercices, food, masses) et des inputs prévus dans le futur (calories exercices et food) pour prévoir les masses dans le futur

In [195]:
class CreateDataset2():
    
    # default values
    PAST = 2
    FUTURE = 1
    
    def __init__(self, dataframe,  past=None, future=None):
        """Constructeur.

        Args:
            dataframe (pd.Dataframe): dataframe des données d'entrée. Index formé de datetimes. Colonnes : 'exo_cals_nets', 'Calories', 'Masse_Totale', 'Masse_Grasse'
            past (int, optional): nombre de jours dans le passé. Defaults to None.
            future (int, optional): nombre de jours dans le futur pour la prévision. Defaults to None
        """
        
        self.data = dataframe
        self.past = self.PAST if past is None else past
        self.future = self.FUTURE if future is None else future
        
    def create_dataset(self):
        """construit et retourne le dataset
        """
        id_start = self.past - 1  # indice rang de départ
        id_stop = len(self.data) - self.future - 1 # indice max 
        n_timesteps = id_stop - id_start
        n_features = self.past * 4 + self.future * 2 # dimension du vecteur input à chaque timestep
        output_dim = (self.future+1) * 2 # dimension vecteur sortie
        
        inputs = np.zeros(shape=(n_timesteps, n_features))  # la sortie sera un np.array de shape n_timesteps
        outputs = np.zeros(shape=(n_timesteps, output_dim))
        
        for i in range(id_start, id_stop):
            # print(id_start, id_stop, i, i-self.past, i+1, i+self.future +1)
            x1 = self.data.iloc[i-self.past+1:i+1,:].to_numpy().flatten()
            # print(x1.shape)
            x2 = self.data.iloc[i+1:i+self.future+1,:2].to_numpy().flatten()
            # print(x2.shape)
            x = np.concatenate([x1, x2])
            inputs[i-id_start] = x
            # print(self.data.iloc[i+1:i+self.future+2,-2:].to_numpy().flatten())
            outputs[i-id_start] = self.data.iloc[i+1:i+self.future+2,-2:].to_numpy().flatten()
            
        df = pd.DataFrame(data = np.concatenate((inputs, outputs), axis=1), index=ds.index[self.past:-self.future])
            
        return inputs, outputs, df

In [196]:
e = CreateDataset2(ds)

In [197]:
inputs, outputs, df = e.create_dataset()

In [198]:
inputs[0]

array([1861.3 ,    0.  ,  107.25,   37.19, 1772.4 ,    0.  ,  104.65,
         33.15, 1894.3 ,    0.  ])

In [199]:
outputs[0]

array([104.35,  33.8 , 104.4 ,  34.34])

In [200]:
ds.tail(12)

,Calories,exo_cals_nets,Masse_Totale,Masse_Grasse
2024-01-22,1714.6,307.409505,72.324167,12.001667
2024-01-23,1656.9,0.000000,72.640000,12.190833
2024-01-24,1731.9,373.559858,72.864167,12.309167
2024-01-25,1803.3,161.198547,73.599167,12.578333
2024-01-26,1778.6,226.307982,73.905833,12.539167
2024-01-27,1775.9,300.393829,73.350000,12.182857
2024-01-28,1821.9,294.128988,73.339167,12.444167
2024-01-29,1795.7,181.483944,73.896667,12.780000
2024-01-30,1680.9,0.000000,73.340000,12.521538
2024-01-31,1682.0,332.372089,73.571538,12.973077


In [201]:
df.tail(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
2024-01-13,1705.0,194.429574,73.260000,12.492500,1747.2,393.320839,72.764167,12.117500,1702.5,0.000000,72.566667,12.447500,72.564167,12.331667
2024-01-14,1747.2,393.320839,72.764167,12.117500,1702.5,0.000000,72.566667,12.447500,1718.3,145.210370,72.564167,12.331667,72.333333,12.560833
2024-01-15,1702.5,0.000000,72.566667,12.447500,1718.3,145.210370,72.564167,12.331667,1684.4,346.402211,72.333333,12.560833,72.593333,12.503333
2024-01-16,1718.3,145.210370,72.564167,12.331667,1684.4,346.402211,72.333333,12.560833,1655.6,396.074120,72.593333,12.503333,72.378462,12.340769
2024-01-17,1684.4,346.402211,72.333333,12.560833,1655.6,396.074120,72.593333,12.503333,1637.2,164.942824,72.378462,12.340769,73.055000,12.607500
2024-01-18,1655.6,396.074120,72.593333,12.503333,1637.2,164.942824,72.378462,12.340769,1661.5,0.000000,73.055000,12.607500,72.494167,12.419167
2024-01-19,1637.2,164.942824,72.378462,12.340769,1661.5,0.000000,73.055000,12.607500,1671.0,331.818404,72.494167,12.419167,72.846667,12.586667
2024-01-20,1661.5,0.000000,73.055000,12.607500,1671.0,331.818404,72.494167,12.419167,1771.0,415.300451,72.846667,12.586667,72.544167,12.618333
2024-01-21,1671.0,331.818404,72.494167,12.419167,1771.0,415.300451,72.846667,12.586667,2014.0,297.844003,72.544167,12.618333,72.324167,12.001667
2024-01-22,1771.0,415.300451,72.846667,12.586667,2014.0,297.844003,72.544167,12.618333,1714.6,307.409505,72.324167,12.001667,72.640000,12.190833
